# Here is a translation of Georgeos' fittingexample.py

'''This script goes along the blog post
"Building powerful image classification models using very little data"
from blog.keras.io.
It uses data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data
In our setup, we:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 12500-13499 in data/train/dogs
- put the dog pictures index 13500-13900 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
```
'''

In [2]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

from keras import backend as K
K.set_image_dim_ordering('th')
import os

Using TensorFlow backend.


In [3]:
os.chdir("/home/bombus/02456-deep-learning/Project/Data/CatsandDogs/")

In [4]:
# dimensions of our images.
img_width, img_height = 150, 150

top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
test_dir = 'data/test'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16



In [13]:
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1. / 255)

    # build the VGG16 network
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_train = model.predict_generator(
        generator, nb_train_samples // batch_size)
    np.save('bottleneck_features_train',
        bottleneck_features_train)

    print("done train")
    generator = datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)
    bottleneck_features_validation = model.predict_generator(
        generator, nb_validation_samples // batch_size)
    np.save('bottleneck_features_validation',
            bottleneck_features_validation)
    print("done valid")



def train_top_model():
    train_data = np.load('bottleneck_features_train.npy')
    train_labels = np.array([0] * (nb_train_samples // 2) + [1] * (nb_train_samples // 2))

    validation_data = np.load('bottleneck_features_validation.npy')
    validation_labels = np.array([0] * (nb_validation_samples // 2) + [1] * (nb_validation_samples // 2))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
    print("starting fitting")
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)
    print("ended fitting and saving")


def predict_image_class(file):
    #load the VGG16 model
    model = applications.VGG16(include_top=False, weights='imagenet')
    #load the picture
    x = load_img(file, target_size=(img_width,img_height))
    x = img_to_array(x)
    x = np.expand_dims(x, axis=0)
    # process the picture through the first network this will be used as input
    #for the top model
    array = model.predict(x)
    #build the top model
    model = Sequential()
    model.add(Flatten(input_shape=array.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # and use the weigths trained before
    model.load_weights(top_model_weights_path)
    class_predicted = model.predict_classes(array)
    if class_predicted==1:
        print("dog")
    else:
        print("cat")


In [7]:
save_bottlebeck_features()
train_top_model()

/home/bombus/anaconda3/lib/python3.6/site-packages/keras/applications/vgg16.py:181: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


Found 2002 images belonging to 2 classes.
done train
Found 800 images belonging to 2 classes.
done valid
starting fitting
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 2s - loss: 0.7000 - acc: 0.7650 - val_loss: 0.3303 - val_acc: 0.8525
Epoch 2/50
2000/2000 [==============================] - 2s - loss: 0.3703 - acc: 0.8520 - val_loss: 0.2540 - val_acc: 0.8950
Epoch 3/50
2000/2000 [==============================] - 2s - loss: 0.2897 - acc: 0.8845 - val_loss: 0.2639 - val_acc: 0.8925
Epoch 4/50
2000/2000 [==============================] - 2s - loss: 0.2614 - acc: 0.8995 - val_loss: 0.3934 - val_acc: 0.8688
Epoch 5/50
2000/2000 [==============================] - 2s - loss: 0.2137 - acc: 0.9095 - val_loss: 0.5365 - val_acc: 0.8313
Epoch 6/50
2000/2000 [==============================] - 2s - loss: 0.2011 - acc: 0.9230 - val_loss: 0.2851 - val_acc: 0.9012
Epoch 7/50
2000/2000 [==============================] - 2s - loss: 0.1982 - acc: 

In [14]:
predict_image_class(test_dir + "/catTest.jpg")
predict_image_class(test_dir + "/dogTest.jpg")

/home/bombus/anaconda3/lib/python3.6/site-packages/keras/applications/vgg16.py:181: UserWarning: You are using the TensorFlow backend, yet you are using the Theano image data format convention (`image_data_format="channels_first"`). For best performance, set `image_data_format="channels_last"` in your Keras config at ~/.keras/keras.json.
  warnings.warn('You are using the TensorFlow backend, yet you '


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 3, None, None)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 64, None, None)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 64, None, None)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, None, None)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 128, None, None)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 128, None, None)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 128, None, None)   0         
__________